In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
import cvxpy as cp
from datetime import datetime

# Parameters
ticker_symbol = "AAPL"
start = "2023-01-01"
today = pd.Timestamp.today().date()
r = 0.05  # Risk-free interest rate
N = 20    # Number of time steps

# Data Fetch
ticker = yf.Ticker(ticker_symbol)
historical_data = ticker.history(start=start, end=today)
closing_prices = historical_data['Close']
option_expirations = ticker.options
chosen_expiry = option_expirations[-1]
options_chain = ticker.option_chain(chosen_expiry)
calls_data = options_chain.calls[['strike', 'lastPrice']]
puts_data = options_chain.puts[['strike', 'lastPrice']]

# Choose one strike price for both Call and Put
strike = float(calls_data['strike'].values[-1])

# Volatility
log_returns = np.log(closing_prices / closing_prices.shift(1)).dropna()
sigma = log_returns.std() * np.sqrt(252)  # Annualized volatility

# Model Parameters
S0 = closing_prices.iloc[-1]
T = (pd.to_datetime(chosen_expiry) - pd.Timestamp.today()).days / 365
dt = T / N
u = np.exp(sigma * np.sqrt(dt))
d = 1 / u
p = (np.exp(r * dt) - d) / (u - d)

# Build Terminal Node Prices
terminal_prices = [S0 * (u ** (N - j)) * (d ** j) for j in range(N + 1)]

# Payoffs
call_payoffs = np.array([max(0, S - strike) for S in terminal_prices])
put_payoffs = np.array([max(0, strike - S) for S in terminal_prices])

# Square Payoff Matrices
call_matrix = call_payoffs.reshape(-1, 1)
call_matrix = np.tile(call_matrix, (1, N + 1))

put_matrix = put_payoffs.reshape(-1, 1)
put_matrix = np.tile(put_matrix, (1, N + 1))

# ==== Nash Equilibrium: CALL Option ====
x_call = cp.Variable(N + 1)
y_call = cp.Variable(N + 1)
v_call = cp.Variable()

constraints_call = [
    cp.sum(x_call) == 1, x_call >= 0,
    cp.sum(y_call) == 1, y_call >= 0,
    call_matrix.T @ x_call >= v_call,
    call_matrix @ y_call <= v_call
]

prob_call = cp.Problem(cp.Minimize(0), constraints_call)
prob_call.solve()

# ==== Nash Equilibrium: PUT Option ====
x_put = cp.Variable(N + 1)
y_put = cp.Variable(N + 1)
v_put = cp.Variable()

constraints_put = [
    cp.sum(x_put) == 1, x_put >= 0,
    cp.sum(y_put) == 1, y_put >= 0,
    put_matrix.T @ x_put >= v_put,
    put_matrix @ y_put <= v_put
]

prob_put = cp.Problem(cp.Minimize(0), constraints_put)
prob_put.solve()

# ==== Output ====
print("====== CALL OPTION GAME ======")
print(f"Strike Price: {strike}")
print(f"Market Call Price: {float(calls_data['lastPrice'].values[-1]):.2f}")
print(f"Nash Game Price (Call): {v_call.value:.2f}\n")

print("====== PUT OPTION GAME ======")
print(f"Strike Price: {strike}")
print(f"Market Put Price: {float(puts_data['lastPrice'].values[-1]):.2f}")
print(f"Nash Game Price (Put): {v_put.value:.2f}\n")

# Optional: Relative Error
call_market = float(calls_data['lastPrice'].values[-1])
put_market = float(puts_data['lastPrice'].values[-1])

print("Relative Error (Call):", round((v_call.value - call_market) / call_market * 100, 2), "%")
print("Relative Error (Put):", round((v_put.value - put_market) / put_market * 100, 2), "%")


====== CALL OPTION GAME ======
Strike Price: 440.0
Market Call Price: 1.84
Nash Game Price (Call): 849.49

====== PUT OPTION GAME ======
Strike Price: 440.0
Market Put Price: 225.05
Nash Game Price (Put): 405.42

Relative Error (Call): 46067.97 %
Relative Error (Put): 80.14 %
